<a href="https://colab.research.google.com/github/guymatz/csci-49376/blob/develop/csci_49376_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount()

Mounted at /content/drive


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=6a09274cb1620b879a9cedbd0b6a41abee0e9feb26475c98d2cd4212a55fa9f2
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql import functions
# Create a Spark Session
ss = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
edges = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/edges.tsv", header=True, sep="\t")
nodes = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/nodes.tsv", header=True, sep="\t")

In [5]:
print(f"Edges: {edges.count()}")
print(f"Nodes: {nodes.count()}")

Edges: 1292203
Nodes: 23036


In [6]:
# 1. For each drug, compute the number of genes
# and the number of diseases associated with the
# drug. Output results with top 5 number of genes in a
# descending order

In [7]:
compounds = edges.filter(
    edges.source.startswith('Compound') &
    (
        edges.target.startswith('Gene') 
        |
        edges.target.startswith('Disease')
    )
)

In [8]:
compounds.sample(.01).show()

+-----------------+--------+-------------------+
|           source|metaedge|             target|
+-----------------+--------+-------------------+
|Compound::DB00773|     CtD| Disease::DOID:1192|
|Compound::DB00860|     CtD| Disease::DOID:3083|
|Compound::DB00445|     CtD| Disease::DOID:1781|
|Compound::DB00359|     CtD|Disease::DOID:12365|
|Compound::DB05294|     CtD| Disease::DOID:1781|
|Compound::DB00620|     CtD| Disease::DOID:9008|
|Compound::DB00457|     CtD|Disease::DOID:10763|
|Compound::DB00514|     CbG|         Gene::1136|
|Compound::DB00619|     CbG|          Gene::759|
|Compound::DB00549|     CbG|         Gene::1557|
|Compound::DB01229|     CbG|          Gene::596|
|Compound::DB01567|     CbG|         Gene::2564|
|Compound::DB00444|     CbG|          Gene::368|
|Compound::DB01142|     CbG|         Gene::3362|
|Compound::DB00694|     CbG|         Gene::9429|
|Compound::DB00972|     CbG|         Gene::1543|
|Compound::DB00571|     CbG|          Gene::155|
|Compound::DB00801| 

In [9]:
c3 = compounds.withColumn("target_type", functions.split(compounds.target, '::').getItem(0))
c4 = c3.drop('metaedge', 'target')
c4.show()

+-----------------+-----------+
|           source|target_type|
+-----------------+-----------+
|Compound::DB00997|    Disease|
|Compound::DB00206|    Disease|
|Compound::DB00960|    Disease|
|Compound::DB00665|    Disease|
|Compound::DB00290|    Disease|
|Compound::DB01232|    Disease|
|Compound::DB00555|    Disease|
|Compound::DB00444|    Disease|
|Compound::DB00860|    Disease|
|Compound::DB00635|    Disease|
|Compound::DB00985|    Disease|
|Compound::DB00808|    Disease|
|Compound::DB01244|    Disease|
|Compound::DB00888|    Disease|
|Compound::DB00591|    Disease|
|Compound::DB00530|    Disease|
|Compound::DB00191|    Disease|
|Compound::DB00620|    Disease|
|Compound::DB00853|    Disease|
|Compound::DB00654|    Disease|
+-----------------+-----------+
only showing top 20 rows



In [19]:
grouped_compounds = c4.groupBy('source','target_type').count()
grouped_compounds.take(5)

[Row(source='Compound::DB01229', target_type='Disease', count=4),
 Row(source='Compound::DB01254', target_type='Disease', count=1),
 Row(source='Compound::DB01014', target_type='Gene', count=35),
 Row(source='Compound::DB00246', target_type='Gene', count=35),
 Row(source='Compound::DB08810', target_type='Gene', count=3)]

In [20]:
#filtered_compounds = compounds.rdd.map(lambda x: ((x.source,"G"), 1) if x.target.startswith("Gene") else ((x.source, "D"), 1))
#filtered_compounds.take(10)
#compounds.rdd.map(lambda x: x.source,"G" if x.target.startswith("Gene") else x.source, "D")
#c = compounds.rdd.map(lambda x: (x.source,"G") if x.target.startswith("Gene") else (x.source, "D"))
grouped_compounds.sort('count','source', ascending=False).take(5)

[Row(source='Compound::DB08865', target_type='Gene', count=585),
 Row(source='Compound::DB01254', target_type='Gene', count=564),
 Row(source='Compound::DB00997', target_type='Gene', count=532),
 Row(source='Compound::DB00570', target_type='Gene', count=523),
 Row(source='Compound::DB00390', target_type='Gene', count=522)]

In [55]:
filtered_compound_counts = filtered_compounds.reduceByKey(lambda a,b:a + b)
filtered_compound_counts.take(5)

[(('Compound::DB00960', 'D'), 1),
 (('Compound::DB00665', 'D'), 1),
 (('Compound::DB01232', 'D'), 1),
 (('Compound::DB00555', 'D'), 3),
 (('Compound::DB00635', 'D'), 17)]

In [59]:
sorted_count = filtered_compound_counts.sortBy(lambda x: x[1], ascending=False)
sorted_count.take(5)

[(('Compound::DB08865', 'G'), 585),
 (('Compound::DB01254', 'G'), 564),
 (('Compound::DB00997', 'G'), 532),
 (('Compound::DB00570', 'G'), 523),
 (('Compound::DB00390', 'G'), 522)]

In [ ]:
# Q2: Compute the number of diseases associated
# with 1, 2, 3, …, n drugs. Output results with the top
# 5 number of diseases in a descending order.
disease_compounds = edges.filter(
    edges.source.startswith('Compound') &
    (
        edges.target.startswith('Disease')
    )

)

In [ ]:
filtered_disease_compounds = disease_compounds.rdd.map(lambda x: (x.target,1))
filtered_disease_compounds.take(5)
filtered_disease_compound_counts = filtered_disease_compounds.reduceByKey(lambda a,b:a + b)
sorted_disease_count = filtered_disease_compound_counts.sortBy(lambda x: x[1], ascending=False)
sorted_disease_count.take(5)

[('Disease::DOID:10763', 70),
 ('Disease::DOID:2531', 53),
 ('Disease::DOID:1612', 43),
 ('Disease::DOID:2841', 40),
 ('Disease::DOID:3393', 38)]

In [ ]:
# Q3: Get the name of drugs that have the top 5 number of genes. Output the results.